# 第10次实验实验报告:整数规划问题
计算机系 计15 2021010803 郭高旭

## 实验目的
- 练习使用python求解整数规划问题
- 掌握使用python求解整数规划问题的方法
- 通过实例使用整数规划模型解决简化的实际问题
- 了解整数规划问题的应用领域

## 第6题：二次指派问题

### 问题分析与算法设计

我们有 $ n $ 个员工和 $ n $ 个城市，每个城市需要指派一名员工。员工之间每月通话的时间矩阵 $ T $ 和城市之间通话费率矩阵 $ C $ 给出如下：

- $ T $ 是一个 $ n \times n $ 的矩阵，其中 $ T[i][j] $ 表示员工 $ i $ 和员工 $ j $ 每月的通话时间（仅上三角部分给出）。
- $ C $ 是一个 $ n \times n $ 的矩阵，其中 $ C[i][j] $ 表示城市 $ i $ 和城市 $ j $ 之间的通话费率（仅下三角部分给出）。

目标是找到一个指派方案，使得总的电话费用最小。

### 算法设计

这是一个典型的二次指派问题，可以使用整数规划方法求解。我们定义决策变量 $ x_{ik} $ 表示员工 $ i $ 被指派到城市 $ k $，目标函数为最小化总的电话费用。约束条件包括每个员工被指派到一个城市、每个城市被指派一个员工、决策变量是0或1。

### 数学模型

定义决策变量：
$$ x_{ik} = \begin{cases} 
1 & \text{如果员工 } i \text{ 被指派到城市 } k \\
0 & \text{否则}
\end{cases} $$

目标函数：
$$ \text{Minimize} \quad \sum_{i=1}^n \sum_{j=1}^n \sum_{k=1}^n \sum_{l=1}^n T[i][j] \cdot C[k][l] \cdot x_{ik} \cdot x_{jl} $$

约束条件：
1. 每个员工被指派到一个城市：
$$ \sum_{k=1}^n x_{ik} = 1 \quad \forall i = 1, \ldots, n $$
2. 每个城市被指派一个员工：
$$ \sum_{i=1}^n x_{ik} = 1 \quad \forall k = 1, \ldots, n $$
3. 决策变量是0或1：
$$ x_{ik} \in \{0, 1\} \quad \forall i, k $$

### 代码实现

In [35]:
import numpy as np
import gurobipy as grb
from gurobipy import Model, GRB, quicksum

# 数据输入
def get_data():
    rec = np.array([
        [0 , 5 , 3 , 7 ,  9 , 3 , 9 , 2 , 9 , 0],
        [7 , 0 , 7 , 8 ,  3 , 2 , 3 , 3 , 5 , 7],
        [4 , 8 , 0 , 9 ,  3 , 5 , 3 , 3 , 9 , 3],
        [6 , 2 , 10, 0 ,  8 , 4 , 1 , 8 , 0 , 4],
        [8 , 6 , 4 , 6 ,  0 , 8 , 8 , 7 , 5 , 9],
        [8 , 5 , 4 , 6 ,  6 , 0 , 4 , 8 , 0 , 3],
        [8 , 6 , 7 , 9 ,  4 , 3 , 0 , 7 , 9 , 5],
        [6 , 8 , 2 , 3 ,  8 , 8 , 6 , 0 , 5 , 5],
        [6 , 3 , 6 , 2 ,  8 , 3 , 7 , 8 , 0 , 5],
        [5 , 6 , 7 , 6 ,  6 , 2 , 8 , 8 , 9 , 0]
    ])
    return rec

# 数据处理
def process_data(rec):
    triu = np.triu(rec)  # 取出矩阵的上三角部分
    tril = np.tril(rec)  # 取出矩阵的下三角部分
    time = triu + triu.T
    price = tril + tril.T
    return time, price

# 模型构建与求解
def solve_quadratic_assignment(time, price):
    N = time.shape[0]
    model = Model("QuadraticAssignmentProblem")

    # 创建决策变量
    x = model.addVars(N, N, vtype=GRB.BINARY, name="x")

    # 设置目标函数
    objective = quicksum(x[i, j] * x[m, n] * time[i, m] * price[j, n]
                         for i in range(N) for j in range(N)
                         for m in range(N) for n in range(N))
    model.setObjective(objective / 2, GRB.MINIMIZE)

    # 添加约束
    model.addConstrs((x.sum(i, '*') == 1 for i in range(N)), "assignEmployee")
    model.addConstrs((x.sum('*', i) == 1 for i in range(N)), "assignCity")

    # 求解模型
    model.optimize()

    return model

# 结果输出
def print_results(model):
    if model.status == GRB.OPTIMAL:
        print('目标函数值是：', model.objVal)
        model.printAttr('X')
        print("指派矩阵为:\n", np.array([[model.getVarByName(f"x[{i},{j}]").X for j in range(10)] for i in range(10)]))
    else:
        print("没有找到最优解")

# 主函数
def main():
    rec = get_data()
    time, price = process_data(rec)
    model = solve_quadratic_assignment(time, price)
    print_results(model)

if __name__ == "__main__":
    main()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 100 columns and 200 nonzeros
Model fingerprint: 0xc0a57822
Model has 3780 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1397.0000000
Presolve time: 0.04s
Presolved: 20 rows, 100 columns, 200 nonzeros
Presolved model has 3880 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective -3.181574e+03, 120 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objecti

### 结果分析与结论

- 使用gurobipy求解上述二次指派问题,得到提示:Optimal solution found (tolerance 1.00e-04).说明求解成功。
- 工人2指派给城市1,工人7指派给城市2,工人4指派给城市3,工人9指派给城市4,工人8指派给城市5,工人5指派给城市6,工人6指派给城市7,工人3指派给城市8,工人1指派给城市9,工人10指派给城市10,每月最小通话费用1142元。

### 意义

- 二次指派问题是一类重要的组合优化问题，有广泛的应用。例如，分配问题、调度问题、指派问题等。
- 二次指派问题的求解方法有很多，例如整数规划、动态规划、贪心算法等。
- 通过对上述二次指派问题的求解，我们可以了解整数规划方法的应用,并且可以解决一些实际问题。



## 第9题：原油采购与加工
### 问题分析与模型建立

根据生活常识,设置优化目标为最大化利润.则可以设决策变量$x_1,x_2,x_3,x_4,x_5$分别代表:甲中原油A的数量,甲中原油B的数量,乙中原油A的数量,乙中原油B的数量,购买原油A的数量.由于原油A的市场价是分段的,所以我们可以把解空间也相应分段,在每一段上求解出最优解,最后取最大值即可.
首先可以列出净售价:
$$
\text{Net Price} = 4800(x_1+x_2)+5600(x_3+x_4)
$$
成本是分段的,可以列出成本函数:
- $ 0 \leq x_5 \leq 500 $:
  $$
    \text{Cost} = 10000x_5
    $$
- $ 500 \le x_5 \leq 1000 $:
  $$
    \text{Cost} = 10000 \times 500 + 8000(x_5-500)=8000x_5+1000000
    $$
- $ 1000 \le x_5 \leq 1500 $:
  $$
    \text{Cost} = 10000 \times 500 + 8000 \times 500 + 6000(x_5-1000)=6000x_5+3000000
    $$

则总利润为:
$$
\text{Profit} = \text{Net Price} - \text{Cost}
$$

约束函数分别为
- 甲中A含量的约束:$x_1\geq x_2$
- 乙中A含量的约束:$2x_3\geq 3x_4$
- 库存约束
  - $x_1+x_3 \leq x_5+500$
  - $x_2+x_4 \leq 1000$

观察可以得知,这是线性规划问题.

### 代码实现与结果

#### 连续规划


In [36]:
import numpy as np
from scipy.optimize import linprog

c1=-np.array([4800,4800,5600,5600,-10000])
c2=-np.array([4800,4800,5600,5600,-8000])
c3=-np.array([4800,4800,5600,5600,-6000])

A_base=np.array([[-1,1,0,0,0],[0,0,-2,3,0],[1,0,1,0,-1],[0,1,0,1,0]])
B=np.array([0,0,500,1000])
bounds_1=[(0,2000),(0,1000),(0,2000),(0,1000),(0,500)]
bounds_2=[(0,2000),(0,1000),(0,2000),(0,1000),(500,1000)]
bounds_3=[(0,2000),(0,1000),(0,2000),(0,1000),(1000,1500)]


q1=linprog(c1,A_ub=A_base,b_ub=B,bounds=bounds_1,method='highs')
q2=linprog(c2,A_ub=A_base,b_ub=B,bounds=bounds_2,method='highs')
q3=linprog(c3,A_ub=A_base,b_ub=B,bounds=bounds_3,method='highs')

print("q1 利润最大值为：",-q1.fun,"最优解为：",q1.x)
print("q2 利润最大值为：",-q2.fun-1000000,"最优解为：",q2.x)
print("q3 利润最大值为：",-q3.fun-3000000,"最优解为：",q3.x)

q1 利润最大值为： 4800000.0 最优解为： [500. 500.  -0.   0.   0.]
q2 利润最大值为： 5000000.0 最优解为： [  -0.   -0. 1500. 1000. 1000.]
q3 利润最大值为： 5000000.0 最优解为： [   0.   -0. 1500. 1000. 1000.]


#### 整数规划

In [37]:
from gurobipy import Model, GRB

# 创建第一个模型
model_1 = Model("profit_maximization_1")
model_1.setParam('OutputFlag', 0)
x1 = model_1.addVars(5, vtype=GRB.INTEGER, name="x1")
objective_1 = 4800 * x1[0] + 4800 * x1[1] + 5600 * x1[2] + 5600 * x1[3] - 10000 * x1[4]
model_1.addConstr(x1[1] - x1[0] <= 0, name="c1_1")
model_1.addConstr(3 * x1[3] - 2 * x1[2] <= 0, name="c2_1")
model_1.addConstr(x1[0] + x1[2] - x1[4] <= 500, name="c3_1")
model_1.addConstr(x1[1] + x1[3] <= 1000, name="c4_1")
model_1.setObjective(objective_1, GRB.MAXIMIZE)
model_1.addConstr(x1[4] >= 0)
model_1.addConstr(x1[4] <= 500)
model_1.optimize()

if model_1.Status == GRB.OPTIMAL:
    print("model_1最优解为：", model_1.objVal)
    for v in model_1.getVars():
        print(v.varName, v.x)

# 创建第二个模型
model_2 = Model("profit_maximization_2")
model_2.setParam('OutputFlag', 0)
x2 = model_2.addVars(5, vtype=GRB.INTEGER, name="x2")
objective_2 = 4800 * x2[0] + 4800 * x2[1] + 5600 * x2[2] + 5600 * x2[3] - 8000 * x2[4] - 1000000
model_2.addConstr(x2[1] - x2[0] <= 0, name="c1_2")
model_2.addConstr(3 * x2[3] - 2 * x2[2] <= 0, name="c2_2")
model_2.addConstr(x2[0] + x2[2] - x2[4] <= 500, name="c3_2")
model_2.addConstr(x2[1] + x2[3] <= 1000, name="c4_2")
model_2.setObjective(objective_2, GRB.MAXIMIZE)
model_2.addConstr(x2[4] >= 500)
model_2.addConstr(x2[4] <= 1000)
model_2.optimize()

if model_2.Status == GRB.OPTIMAL:
    print("model_2最优解为：", model_2.objVal)
    for v in model_2.getVars():
        print(v.varName, v.x)

# 创建第三个模型
model_3 = Model("profit_maximization_3")
model_3.setParam('OutputFlag', 0)
x3 = model_3.addVars(5, vtype=GRB.INTEGER, name="x3")
objective_3 = 4800 * x3[0] + 4800 * x3[1] + 5600 * x3[2] + 5600 * x3[3] - 6000 * x3[4] - 3000000
model_3.addConstr(x3[1] - x3[0] <= 0, name="c1_3")
model_3.addConstr(3 * x3[3] - 2 * x3[2] <= 0, name="c2_3")
model_3.addConstr(x3[0] + x3[2] - x3[4] <= 500, name="c3_3")
model_3.addConstr(x3[1] + x3[3] <= 1000, name="c4_3")
model_3.setObjective(objective_3, GRB.MAXIMIZE)
model_3.addConstr(x3[4] >= 1000)
model_3.addConstr(x3[4] <= 1500)
model_3.optimize()

if model_3.Status == GRB.OPTIMAL:
    print("model_3最优解为：", model_3.objVal)
    for v in model_3.getVars():
        print(v.varName, v.x)


model_1最优解为： 4800000.0
x1[0] 500.0
x1[1] 500.0
x1[2] 0.0
x1[3] -0.0
x1[4] -0.0
model_2最优解为： 5000000.0
x2[0] 0.0
x2[1] -0.0
x2[2] 1500.0
x2[3] 1000.0
x2[4] 1000.0
model_3最优解为： 5000000.0
x3[0] 0.0
x3[1] -0.0
x3[2] 1500.0
x3[3] 1000.0
x3[4] 1000.0


### 结果分析

- 可以看到,连续规划的解就是整数规划的解,这是因为连续规划的解本身就全是整数,所以整数规划的解和连续规划的解是一样的.
- 两种方法得到的最优解都是$x_1=0,x_2=0,x_3=1500,x_4=1000,x_5=1000$,最大利润为$5\times 10^6$.

### 结论

- 购买原油A的数量为1000,甲中原油A的数量为0,甲中原油B的数量为0,乙中原油A的数量为1500t,乙中原油B的数量为1000t,最大利润为5000000元.

### 意义

- 通过对原油采购与加工问题的求解,我们可以了解线性规划和整数规划的应用,并且可以解决一些实际问题.

## 第12题：下料问题

### 问题分析与模型建立

问题是在给定切割方案的情况下，求解最优的下料方法。我们可以将问题建模为整数规划问题，定义决策变量 $ x_i ,i=1,2,\ldots,4 $ 表示第 $ i $ 个下料方案消耗的张数.$y_1,y_2$分别为不配套的罐身与罐底数,z表示生产的易拉罐数.由于切割模式已经确定,因此可以总结出4种冲压模式的特征如表4:
| 模式 | 罐身个数 | 罐底个数 | 余料损失($cm^2$) |冲压时间|
| --- | --- | --- | --- | --- |
| 1 | 1 | 10 | 222.57 |1.5|
| 2 | 2 | 4 | 183.3 |2|
| 3 | 0 | 16 | 261.84 |1|
| 4 | 4 | 5 | 169.5 |3|

根据上表可以列出总损失(面积)
$$
\text{Loss} = 222.57x_1+183.3x_2+261.84x_3+169.5x_4+157.1y_1+19.6y_2
$$

故总利润(优化目标)为
$$
\text{Profit} = 0.1z-0.001\text{Loss}
$$

约束条件为
- 原料约束
  $$
  x_1+x_2+x_3 \leq 50000 \\
    x_4 \leq 20000
  $$
- 时间约束
  $$
  1.5x_1+2x_2+x_3+3x_4 \leq 144000
  $$
- 罐身罐底约束:$ z = min((10x_1+4x_2+16x_3+5x_4)/2,x_1+2x_2+4x_4)$
  - 上面就等价于下面两个不等式
  - $z \leq (10x_1+4x_2+16x_3+5x_4)/2$
  - $z \leq x_1+2x_2+4x_4$
  - 此外还有
  - $y_2=10x_1+4x_2+16x_3+5x_4 - 2z$
  - $y_1=x_1+2x_2+4x_4 - z$

### 代码实现与结果

In [38]:
from gurobipy import Model, GRB

# 创建一个新的模型
model = Model("cutting_plan")

# 创建决策变量
x1 = model.addVar(vtype=GRB.INTEGER, name="x1")
x2 = model.addVar(vtype=GRB.INTEGER, name="x2")
x3 = model.addVar(vtype=GRB.INTEGER, name="x3")
x4 = model.addVar(vtype=GRB.INTEGER, name="x4")
y1 = model.addVar(vtype=GRB.CONTINUOUS, name="y1")
y2 = model.addVar(vtype=GRB.CONTINUOUS, name="y2")
z = model.addVar(vtype=GRB.CONTINUOUS, name="z")

# 定义总损失（面积）
loss = 222.57 * x1 + 183.3 * x2 + 261.84 * x3 + 169.5 * x4 + 157.1 * y1 + 19.6 * y2

# 定义总利润
profit = 0.1 * z - 0.001 * loss

# 设置目标函数为最大化总利润
model.setObjective(profit, GRB.MAXIMIZE)

# 添加约束条件
model.addConstr(x1 + x2 + x3 <= 50000, "raw_material_1")
model.addConstr(x4 <= 20000, "raw_material_2")
model.addConstr(1.5 * x1 + 2 * x2 + x3 + 3 * x4 <= 144000, "time")

# 罐身罐底约束
model.addConstr(z <= (10 * x1 + 4 * x2 + 16 * x3 + 5 * x4) / 2, "body_bottom_1")
model.addConstr(z <= x1 + 2 * x2 + 4 * x4, "body_bottom_2")

# y1 和 y2 的约束
model.addConstr(y2 == 10 * x1 + 4 * x2 + 16 * x3 + 5 * x4 - 2 * z, "y2_constraint")
model.addConstr(y1 == x1 + 2 * x2 + 4 * x4 - z, "y1_constraint")

# 优化模型
model.optimize()

# 输出最优解
if model.Status == GRB.OPTIMAL:
    print("最优解为：")
    print("Profit:", model.ObjVal)
    for v in model.getVars():
        print(v.varName, v.x)
else:
    print("没有找到最优解")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 7 columns and 28 nonzeros
Model fingerprint: 0x3eac3ea5
Variable types: 3 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-02, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 1e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 18 nonzeros
Variable types: 3 continuous, 4 integer (0 binary)

Root relaxation: objective 4.298188e+03, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    

### 结果分析

上述模型在guropi求解后成功得到最优解
最优解为：
Profit: 4298.1875
x1 -0.0
x2 40125.0
x3 3750.0
x4 20000.0
y1 0.0
y2 0.0
z 160250.0

### 结论
第2,3,4种下料方案分别消耗40125,3750,20000张铝板,生产160250个易拉罐,总利润为4298.1875元.

### 意义
通过对上述问题的求解,可以指导易拉罐生产厂家如何选择最优的下料方案,从而提高生产效率,降低生产成本.


